## gensim으로 네이버 기사 토픽 모델링 해보기

> 토픽 모델링을 적용하기 위해 텍스트를 처리합니다.

> 토픽 모델링 라이브러리인 gensim을 사용해봅니다.

### 1. 토픽 모델링을 위한 라이브러리 불러오기

In [1]:
from tqdm import tqdm_notebook # progress bar
from konlpy.tag import Mecab # Mecab, Okt 등 형태소 분석기 불러오기
import string # 특수문자
import warnings # 경고 알림 제거를 위한 라이브러리
from gensim import corpora # gensim에서 사용하는 vectorizer 모듈과, LDA model을 불러온다.
from gensim import models



import numpy as np
import re
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings("ignore", category=DeprecationWarning) # 경고 알림이 뜨면 모두 무시합니다.

mecab = Mecab(r"C:/mecab/mecab-ko-dic")

### 2. 텍스트 전처리 함수 만들기

In [2]:
def read_documents(input_file_name):
    
    corpus = []
    
    # pk 파일을 읽어서 리스트로 변환하여 돌려줌.
    
    with open(input_file_name, 'rb') as f:
        temp_corpus = pickle.load(f)

    for page in temp_corpus:
        corpus += page
    
    return corpus

def text_cleaning(docs):
    # 한국어를 제외한 글자를 제거하는 함수를 편의를 위해 조금 변형해보자.
    cleaned_docs = []

    for doc in docs:
        temp_doc = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", doc)
        cleaned_docs.append(temp_doc)    
    return cleaned_docs

def define_stopwords(path):
    
    SW = set()
    # 불용어를 추가하는 방법 1.
    # 특수 문자를 추가해보자.
    for i in string.punctuation:
        SW.add(i)

    # 불용어를 추가하는 방법 2.
    # stopwords-ko.txt에 직접 추가
    
    with open(path, encoding='utf-8') as f:
        for word in f:
            SW.add(word)

    return SW


def text_tokenizing(corpus, tokenizer):
    # 명사 추출 / 형태소 분석 두 가지를 선택할 수 있게 만들어주는 함수를 만들어보자.
    token_corpus = []
    # tqdm을 사용하여 진행 과정을 볼 수 있게 만들어보자.
    if tokenizer == "noun":
        for n in tqdm_notebook(range(len(corpus)), desc="Preprocessing"):
            token_text = mecab.nouns(corpus[n])
            token_text = [word for word in token_text if word not in SW and len(word) > 1]

            token_corpus.append(token_text)

    elif tokenizer == "morph":
        for n in tqdm_notebook(range(len(corpus)), desc="Preprocessing"):
            token_text = mecab.morphs(corpus[n])
            token_text = [word for word in token_text if word not in SW and len(word) > 1]
            token_corpus.append(token_text)

    elif tokenizer == "word":
        for n in tqdm_notebook(range(len(corpus)), desc="Preprocessing"):
            token_text = corpus[n].split()
            token_test = [word for word in token_text if word not in SW and len(word) > 1]
            token_corpus.append(token_text)


    return token_corpus

# 함수를 불러오는 (메인) 코드.
input_file_name = "./naver_news_content.pk"
documents = read_documents(input_file_name)
SW = define_stopwords("./stopwords-ko.txt")
cleaned_text = text_cleaning(documents)
tokenized_text = text_tokenizing(cleaned_text, tokenizer="noun") #tokenizer= "noun" or "morph" or "word"

Preprocessing:   0%|          | 0/8 [00:00<?, ?it/s]

문서 읽기의 과정은 앞서 단어 임베딩의 경우와 다르지 않다. 다음 과정은 문서-단어 행렬을 만드는 과정이다.

In [3]:
# 결과 확인.
print(tokenized_text[0])

['본문', '내용', '플레이어', '플레이어', '오류', '우회', '함수', '추가', '디지털', '교육', '실습', '플랫', '엘리스', '대표', '김재원', '무료', '수강', '가능', '디지털', '레딧', '교육', '모집', '디지털', '레딧', '고용', '노동부', '청년', '장년', '구직자', '디지털', '기초', '역량', '강화', '마련', '직업훈련', '일환', '코딩', '데이터', '개발', '디지털', '기술', '분야', '입문', '과정', '교육', '정부', '전체', '교육비', '지원', '이수', '자부', '담비', '환급', '엘리스', '플랫', '기술력', '운영', '노하우', '집약', '괄목', '성과', '지난해', '교육', '대비', '이상', '증가', '교육', '만족도', '만점', '기록', '증가세', '엘리스', '교육', '과정', '특징', '실습', '중심', '교육', '필요', '요소', '최소', '동기', '부여', '학습', '효과', '극대', '로그인', '코딩', '학습', '가능', '화면', '전환', '화면', '영상', '코드', '작성', '검토', '클릭', '작성', '코드', '확인', '현직', '개발', '신속', '응답', '교육', '이해도', '엘리스', '참여', '교육', '업체', '최대', '규모', '과목', '운영', '교육', '역량', '전문', '파이썬', '인공지능', '분야', '코딩', '경험', '입문', '과정', '고급', '과정', '구성', '과목', '자유', '선택', '세부', '과목', '파이썬', '기초', '프로그래밍', '기초', '개발', '페이지', '제작', '파이썬', '데이터', '분석', '데이터', '분석', '인공지능', '수학', '심화', '머신', '러닝', '러닝', '코딩', '테스트', '대비', '알고리즘', '리눅스', '추천', '교육', '과정', '

### 3. 토픽 모델링에 사용할 함수들 확인하기

In [4]:
# 문서-단어 행렬 만들기
# 어휘(vocabulary) 학습
dictionary = corpora.Dictionary(tokenized_text)

# 문서-단어 행렬(document-term matrix) 생성
corpus = [dictionary.doc2bow(text) for text in tokenized_text]


In [5]:
# Dictionary 확인
print(dictionary)

Dictionary(570 unique tokens: ['가능', '강의', '강화', '개발', '개발자']...)


In [6]:
# corpus 확인
corpus[0][:5]

[(0, 2), (1, 1), (2, 1), (3, 3), (4, 1)]

In [7]:
# TFIDF 문서-단어 행렬 생성
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
corpus_tfidf[0][:5]

[(0, 0.03987963511819594),
 (1, 0.04804341196577365),
 (2, 0.03257688787974344),
 (3, 0.05981945267729392),
 (4, 0.06798322952487162)]

In [8]:
# LDA model 만들기
model = models.ldamodel.LdaModel(corpus, num_topics=3, id2word=dictionary)

In [9]:
# LDA 결과 확인
model.show_topic(0, 10)

[('데이터', 0.022270618),
 ('교육', 0.020872094),
 ('디지털', 0.0137061225),
 ('과정', 0.01287343),
 ('정보', 0.011400656),
 ('활용', 0.010883731),
 ('기술', 0.009694057),
 ('코딩', 0.008880244),
 ('엘리스', 0.00881647),
 ('스노우', 0.008197701)]

### 4. 토픽 모델링을 추가하여 코드 완성하기

In [11]:
# 토픽 개수, 키워드 개수를 정해주는 변수를 추가.
NUM_TOPICS = 3

NUM_TOPIC_WORDS = 30

def build_doc_term_mat(documents):
    # 문서-단어 행렬 만들어주는 함수.
    print("Building document-term matrix.")
    dictionary = corpora.Dictionary(documents)
    corpus = [dictionary.doc2bow(document) for document in documents]

    return corpus, dictionary


def print_topic_words(model):

    # 토픽 모델링 결과를 출력해 주는 함수.
    print("\nPrinting topic words.\n")

    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))

        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
        print("\n")

# document-term matrix를 만들고,
corpus, dictionary = build_doc_term_mat(tokenized_text)
# LDA를 실행.
model = models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha="auto", eta="auto")
# 결과를 출력.
print_topic_words(model)

Building document-term matrix.

Printing topic words.

Topic ID: 0
	정보	0.017278097569942474
	공시	0.011852903291583061
	비교	0.011243972927331924
	데이터	0.011179478839039803
	기업	0.010392727330327034
	가능	0.009067007340490818
	회계	0.008189734071493149
	재무	0.0072992411442101
	활용	0.0070260255597531796
	교육	0.007013632450252771
	마련	0.006682110484689474
	이용자	0.006569594144821167
	관련	0.00648503890261054
	도입	0.006227569188922644
	재무제표	0.005903353914618492
	디지털	0.005888347513973713
	기술	0.005814040079712868
	근거	0.005736048799008131
	제도	0.0054870122112333775
	상황	0.005426167044788599
	과정	0.005347718019038439
	관계자	0.005344726145267487
	배경	0.0052069709636271
	이해	0.005082505289465189
	내용	0.005048374645411968
	본문	0.004944810178130865
	우리	0.004816862288862467
	커뮤니티	0.0047545116394758224
	기관	0.004609895870089531
	구체	0.004560282453894615


Topic ID: 1
	교육	0.03204751014709473
	데이터	0.021908359602093697
	과정	0.020246613770723343
	디지털	0.01766597479581833
	엘리스	0.013633333146572113
	기술	0.010847952216863632
	코딩	0.009478

### 5. pyLDAvis를 통한 토픽 모델링 결과 시각화하기

In [12]:
# pyLDAvis 불러오기
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis


# pyLDAvis를 jupyter notebook에서 실행할 수 있게 활성화.
pyLDAvis.enable_notebook()

# pyLDAvis 실행.
data = gensimvis.prepare(model,corpus,dictionary)
data

C:\Users\jx778\anaconda3\envs\test\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.020037 -0.006642       1        1  47.944692
2     -0.032343  0.005677       2        1  31.253832
0      0.052380  0.000965       3        1  20.801476, topic_info=    Term       Freq      Total Category  logprob  loglift
238   정보  16.000000  16.000000  Default  30.0000  30.0000
297   공시   8.000000   8.000000  Default  29.0000  29.0000
185   비교   8.000000   8.000000  Default  28.0000  28.0000
436   회계   7.000000   7.000000  Default  27.0000  27.0000
402   재무   5.000000   5.000000  Default  26.0000  26.0000
..   ...        ...        ...      ...      ...      ...
37   디지털   2.337039  27.959145   Topic3  -5.1348  -0.9117
28    내용   2.003660  14.995681   Topic3  -5.2887  -0.4426
13    과정   2.122467  32.334727   Topic3  -5.2311  -1.1534
50    본문   1.962556  13.925093   Topic3  -5.3094  -0.3893
315   기관   1.829631   9.497080   Topic3  -5.3796  -0.0767

[196 rows x 6 columns], token_table=      Topic      Freq Term
term                      
0         1  0.447978   가능
0         2  0.223989   가능
0         3  0.298652   가능
288       1  0.310112   감독
288       3  0.310112   감독
...     ...       ...  ...
436       2  0.137387   회계
436       3  0.412162   회계
137       1  0.482425   효과
137       2  0.361818   효과
137       3  0.120606   효과

[345 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1])